### Splink Notes

[(sp)link to the docs](https://moj-analytical-services.github.io/splink/)

Cleaning Requirements
- columns shouldn't be correlated
  - profile data with `profile_columns`
- strings are cleaned
- matching columns have same name in both datasets

Pros
- good charts, eg. you can see a match weights waterfall charts to see how column comparison contributes to the match score, cluster charts, comparison vectors etc.
- you can see the comparison levels of match probability - shows the match weight of exact matches versus match weights of levenstein distance <= 2, versus all other comparisons, numeric columns might have just 2 match levels (exact match versus not match)
- good charts makes the model much more intuitive, plant name should likely have a higher match weight than fuel type
- built in data profile
- very responsive on GitHub and and good upkeep of the tool

Cons
- columns needed to be compared directly because the names need to match? Idea: maybe you could try to rename columns that have a lookup dictionary - fuel_type_code_pudl_energy_source_code to relate FTCP and energy source code
- can't have dependencies in columns - seems like it doesn't do as well with dependencies in records
- how does it handle transitivity in the records? if FERC_A links to EIA_B then FERC_A also links to EIA_A
- how will it handle data on the EIA side that isn't going to be linked to a FERC record?
- not the best documentation - had to pick through the source code
- the error messages aren't very helpful or there is a lack of error messaging

Questions:
- Would this work best for connecting the FERC plants and assigning `plant_id_pudl`? Could do a dedupe maybe?
   

Steps:
- profile data
- clean values and columns
- train u values - `train_u_using_random_sampling`
    - u value: what is the probability of collisions even when records don't match? you can get good estimates of the u values just with comparing random samples
- train m values - `train_m_using_expectation_maximisation`, one EM iteration for all blocking rules (just report year?)
    - m value: reports the data quality: amongst matches how often do we get collisions?

In [ ]:
import pandas as pd
from splink.duckdb.duckdb_linker import DuckDBLinker
import splink.duckdb.duckdb_comparison_library as cl
import splink.duckdb.duckdb_comparison_level_library as cll
import sqlalchemy as sa

import pudl

In [ ]:
pudl_engine = sa.create_engine(pudl.workspace.setup.get_defaults()['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine, freq="AS")

### Load the Data
Start with the input data to Panda. 
- EIA distinct plant parts data filtered to just gens for 2020
- Full FERC 

Then try without breaking apart by plant part.

- do the missingness analysis early, get rid of columns with lots of nulls
- rename columns that don't match (fuel type)
- filter for just 2020 and generators
- get rid of dependent columns
- do the link

Read in pickled version of plant parts list or generate a fresh one

In [ ]:
# if you have it, read in pickled dataframe
eia_df = pd.read_pickle("plant_parts_eia_distinct.pkl.gz")

In [ ]:
# make ppl distinct for Panda
# this was adapted from the RMI repo
# takes as input a non-distinct (includes non true grans) ppl
def get_plant_parts_distinct(plant_parts_eia):
    """Get the EIA plant-parts with only the unique granularities."""
    # We want only the records of the EIA plant-parts that are "true
    # granularies" and those which are not duplicates based on their
    # ownership  so the model doesn't get confused as to which option to
    # pick if there are many records with duplicate data
    plant_parts_eia = plant_parts_eia.assign(
        plant_id_report_year_util_id=lambda x: x.plant_id_report_year
        + "_"
        + x.utility_id_pudl.map(str)
    ).astype({"installation_year": "float"})
    plant_parts_distinct = plant_parts_eia[
        (plant_parts_eia["true_gran"]) & (~plant_parts_eia["ownership_dupe"])
    ]
    return plant_parts_distinct

In [ ]:
"""
# else, generate a new PPE and make it distinct
plant_parts_eia = pudl_out.plant_parts_eia()
# a little patch, this might not be needed anymore
plant_parts_eia = plant_parts_eia[~plant_parts_eia.index.duplicated(keep="first")]
eia_df = get_plant_parts_distinct(plant_parts_eia)
"""

Pick up here after generating the PPE

In [ ]:
# currently df is intended to be the distinct plant parts list
def add_utility_name(df, pudl_engine):
    # join on utility_name_eia
    eia_util = pd.read_sql("utilities_eia", pudl_engine)
    eia_util = eia_util.set_index('utility_id_eia')['utility_name_eia']
    non_null_df = df[~(df.utility_id_eia.isnull())]
    non_null_df = non_null_df.merge(eia_util, how="left", left_on='utility_id_eia', right_index=True, validate="m:1")
    df_util = pd.concat([non_null_df, df[df.utility_id_eia.isnull()]])
    df = df_util.reindex(df.index)
    
    return df_util

Instead of filtering for 2020 data, maybe it would work better if this was built in as a blocking rule. Ideally it would be good to have no other blocking rules besides report_year. 

In [ ]:
eia_df = eia_df[eia_df.report_year == 2020]
eia_df = eia_df[eia_df.plant_part == "plant_gen"]
eia_df = add_utility_name(eia_df, pudl_engine)

In [ ]:
# not useful for matching, drop now to clean up missingness chart
extra_cols = ["appro_part_label",
              "appro_record_id_eia",
              "fraction_owned",
              "ownership",
              "ownership_dupe",
              "plant_part",
              "plant_id_report_year",
              "plant_id_report_year_util_id",
              "plant_part_id_eia",
              "record_count",
              "true_gran"]
eia_df = eia_df.drop(columns=extra_cols)

In [ ]:
# read in the cleaned ferc table if you have it
ferc_df = pd.read_pickle("full_ferc.pkl")

In [ ]:
# or generate a fresh one (this is from RMI repo)
def get_ferc_plants(pudl_out):
    fbp_cols_to_use = [
        "report_year",
        "utility_id_ferc1",
        "plant_name_ferc1",
        "utility_id_pudl",
        "fuel_cost",
        "fuel_mmbtu",
        "primary_fuel_by_mmbtu",
    ]
    plants_ferc1_df = (
        pudl_out.plants_all_ferc1()
        .merge(
            pudl_out.fbp_ferc1()[fbp_cols_to_use],
            on=[
                "report_year",
                "utility_id_ferc1",
                "utility_id_pudl",
                "plant_name_ferc1",
            ],
            how="left",
        )
        .pipe(pudl.helpers.convert_cols_dtypes, "ferc1")
        .assign(
            installation_year=lambda x: (
                x.installation_year.astype("float")
            ),  # need for comparison vectors
            plant_id_report_year=lambda x: (
                x.plant_id_pudl.map(str) + "_" + x.report_year.map(str)
            ),
            plant_id_report_year_util_id=lambda x: (
                x.plant_id_report_year + "_" + x.utility_id_pudl.map(str)
            ),
            fuel_cost_per_mmbtu=lambda x: (x.fuel_cost / x.fuel_mmbtu),
            heat_rate_mmbtu_mwh=lambda x: (x.fuel_mmbtu / x.net_generation_mwh),
        )
        .rename(
            columns={
                "record_id": "record_id_ferc1",
                "opex_plants": "opex_plant",
                "fuel_cost": "total_fuel_cost",
                "fuel_mmbtu": "total_mmbtu",
                "opex_fuel_per_mwh": "fuel_cost_per_mwh",
                "primary_fuel_by_mmbtu": "fuel_type_code_pudl",
            }
        )
        .set_index("record_id_ferc1")
    )
    return plants_ferc1_df

In [ ]:
# ferc_df = get_ferc_plants(pudl_out)

In [ ]:
ferc_df = ferc_df[ferc_df.report_year == 2020]

In [ ]:
linker_eia = DuckDBLinker(eia_df)
linker_ferc = DuckDBLinker(ferc_df)

In [ ]:
linker_eia.missingness_chart()

In [ ]:
# drop cols that are more than 80% null
percent_null = eia_df.isnull().sum() / len(eia_df)
cols_to_drop = list(set(percent_null[percent_null >= .8].index))

In [ ]:
eia_df = eia_df.drop(columns=cols_to_drop)

In [ ]:
linker_ferc.missingness_chart()

In [ ]:
eia_df = eia_df.rename(columns={"plant_name_eia": "plant_name", "utility_name_eia": "utility_name"})
ferc_df = ferc_df.rename(columns={"plant_name_ferc1": "plant_name", "utility_name_ferc1": "utility_name"})
shared_cols = list(set(eia_df.columns) & set(ferc_df.columns))

In [ ]:
shared_cols

TODO: look at `compare_multiple_columns_to_single_column_level`. I think this would allow us to compare columns that don't match (only for exact match but could make own function that does a string distance). I think the non-exact match could be done by specifying a distance function for a level with `DistanceFunctionLevelBase` (or one of the specific functions). Added in [this PR](https://github.com/moj-analytical-services/splink/pull/720)

Since we are only using columns that match, we can naively have `plant_type` on the FERC side map to `technology_description` on the EIA side. This is really not a very good mapping, but for now let's not mess with the columns too much, in a future iteration we could concatenate string columns together into one column or make a dictionary mapping plant type to technology description.

Note: this naive mapping might be making results worse in this baseline model, but I wanted a column representing the plant type to make it in here.

In [ ]:
ferc_df.plant_type.isnull().value_counts()

In [ ]:
ferc_df = ferc_df.rename(columns={"plant_type": "technology_description"})
shared_cols = list(set(eia_df.columns) & set(ferc_df.columns))

In [ ]:
# splink doesn't do well when columns are dependent
# some of these columns seem somewhat dependent (technology_description and capacity_mw) but let's just try
shared_cols

In [ ]:
# it's not actually necessary for all columns to match for splink to work
# it's just cleaner to look at the dataframes
ferc_df = ferc_df[shared_cols]
eia_df = eia_df[shared_cols]

In [ ]:
eia_df.dtypes

In [ ]:
eia_df = eia_df.astype({
    "technology_description": "string", 
    "installation_year": "Int64", 
    "construction_year": "Int64",
    "capacity_mw": "float64",
    "utility_name": "string",
    "plant_name": "string",
    "fuel_type_code_pudl": "string"})
eia_df.dtypes

In [ ]:
ferc_df = ferc_df.astype(eia_df.dtypes)

In [ ]:
ferc_df.dtypes

In [ ]:
# basic column string clean up, could do something more thorough later
# technology description really doesn't map very well between FERC and EIA right now
str_cols = ["utility_name", "plant_name", "technology_description"]
eia_df[str_cols] = eia_df[str_cols].apply(lambda x: x.str.strip().str.lower().str.replace(" ", "_"))
ferc_df[str_cols] = ferc_df[str_cols].apply(lambda x: x.str.strip().str.lower().str.replace(" ", "_"))

Since 0 is the most common `capacity_mw` value on the FERC side, replace 0 and negative `capacity_mw` values on the FERC side with nans. On the EIA side, `capacity_mw` values are filled/aggregated from other plant parts within the plant. There are still a few 0 values on the EIA side but not as many as FERC. Even if a 0 capacity value on the FERC side means the record represents a plant part without capacity, then it's better for this to be represented with a null value.

Not completely sure if this is the right thing to do. 

In [ ]:
len(eia_df[eia_df.capacity_mw == 0])

In [ ]:
ferc_df.loc[ferc_df.capacity_mw <= 0, "capacity_mw"] = None
ferc_df = ferc_df.round({"capacity_mw": 2})

In [ ]:
eia_df.loc[eia_df.capacity_mw < 0, "capacity_mw"] = None
eia_df = eia_df.round({"capacity_mw": 2})

In [ ]:
# need to make the record id index a column for splink to have a unique id col
eia_df["record_id"] = eia_df.index
ferc_df["record_id"] = ferc_df.index

Arguments like `additional_columns_to_retain` and `retain_matching_columns` slow down matching but are nice to have when looking at end results. Maybe drop these arguments and join the columns back on if training is too slow.

In [ ]:
settings_dict = {"link_type": "link_only",
                 "unique_id_column_name": "record_id",
                 "additional_columns_to_retain": ["plant_id_pudl", "utility_id_pudl"]}

In [ ]:
linker = DuckDBLinker([ferc_df, eia_df], settings_dict=settings_dict)

### Explore the Data

In [ ]:
from altair.vegalite.v4.display import VegaLite

In [ ]:
# taken from the splink charts.py module. This is janky.
class VegaliteNoValidate(VegaLite):
        def _validate(self):
            pass

def display_chart(spec):

        return VegaliteNoValidate(spec)

In [ ]:
display_chart(linker.missingness_chart(input_dataset="ferc_df"))

In [ ]:
display_chart(linker.missingness_chart(input_dataset="eia_df"))

It doesn't work to do `profile_columns` with the linker that has both dataframes (I think some sort of SQL type conversion error). So for now I separated out the dataframes into two different linkers.

These charts show skew in the fields we're matching on. We might want to use term frequency adjustments in our model to weight more likely matches (like PG&E) less heavily than less likely matches. I think the term frequency adjustments just normalize the match weight by the frequency of the term in the column.

Columns to use `term_frequency_adjustments`: `utility_name`, `technology_description`, `fuel_type_code_pudl` (experiment with this parameter)

In [ ]:
linker_ferc = DuckDBLinker(ferc_df)
linker_eia = DuckDBLinker(eia_df)

In [ ]:
linker_ferc.profile_columns([
    "plant_name", 
    "utility_name", 
    "technology_description", 
    "installation_year", 
    "construction_year", 
    "capacity_mw", 
    "fuel_type_code_pudl"])

In [ ]:
linker_eia.profile_columns([
    "plant_name", 
    "utility_name", 
    "technology_description", 
    "installation_year", 
    "construction_year", 
    "capacity_mw", 
    "fuel_type_code_pudl"])

In [ ]:
# since we don't have any blocking rules, the number of possible matches is the lengths multiplied
len(eia_df) * len(ferc_df)

### Define Comparisons

Is there a good way to tune these thresholds for the similarity metrics with splink?

Panda was using Jaccard distance, which measures the overlap between sets of characters (and hopefully it wasn't using words). It's probably more applicable to use Levenshtein distance to get a measure of how much editing needs to be done between the strings.

I hope this is using characters not words but I should double check.

It seems like most of the built in comparisons are meant for strings.

Experiment with adding and removing `term_frequency_adjustment`. Maybe add for `utility_name` because of the skew on the FERC side?

In [ ]:
# try and add more thresholds too
print(cl.levenshtein_at_thresholds("plant_name", [5]).human_readable_description)

You can define Comparisons with a dictionary.

In [ ]:
from splink.comparison import Comparison

In [ ]:
capacity_comparison = {
    "output_column_name": "capacity_mw",
    "comparison_levels": [
        cll.null_level("capacity_mw"),
        cll.percentage_difference_level("capacity_mw", 0.0 + 1e-4),  # could add an exact match level too
        cll.percentage_difference_level("capacity_mw", 0.1 + 1e-4), # need the 1e-4?
        cll.percentage_difference_level("capacity_mw", 0.2 + 1e-4),
        cll.else_level(),
    ],
    "comparison_description": "0% different vs. 10% different vs. 20% different vs. anything else"
}

In [ ]:
print(Comparison(capacity_comparison).human_readable_description)

Do comparison levels with less than or equal formats need to be exclusive of each other? (seems like no based on the `percentage_difference_level` built in level above.

https://moj-analytical-services.github.io/splink/demos/example_transactions.html

In [ ]:
def get_within_n_years_str(col_name, n):
    return f"abs({col_name}_r - {col_name}_l) <= {n}"
 
installation_year_comparison = {
    "output_column_name": "installation_year",
    "comparison_levels": [
        cll.null_level("installation_year"),
        cll.exact_match_level("installation_year"),
        {"sql_condition": get_within_n_years_str("installation_year", 1), "label_for_charts": "<= 1 year diff"},
        {"sql_condition": get_within_n_years_str("installation_year", 2), "label_for_charts": "<= 2 years diff"},
        cll.else_level()
    ],
    "comparison_description": "Number of years apart"
}

construction_year_comparison = {
    "output_column_name": "construction_year",
    "comparison_levels": [
        cll.null_level("construction_year"),
        cll.exact_match_level("construction_year"),
        {"sql_condition": get_within_n_years_str("construction_year", 1), "label_for_charts": "<= 1 year diff"},
        {"sql_condition": get_within_n_years_str("construction_year", 2), "label_for_charts": "<= 2 years diff"},
        cll.else_level()
    ],
    "comparison_description": "Number of years apart"
}



In [ ]:
print(Comparison(installation_year_comparison).human_readable_description)
print(Comparison(construction_year_comparison).human_readable_description)

Add `term_frequency_adjustments` to other columns?

Leaving off `technology_description` for now. The match is too bad so some column adjustments need to be made.

In [ ]:
settings_dict.update({
    "comparisons": [
        cl.levenshtein_at_thresholds("plant_name", [2, 5]),
        cl.levenshtein_at_thresholds("utility_name", [2, 5]),
        construction_year_comparison,
        installation_year_comparison,
        capacity_comparison,
        cl.exact_match("fuel_type_code_pudl", term_frequency_adjustments=True),
    ],
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": False,
    }
)

In [ ]:
linker = DuckDBLinker([ferc_df, eia_df], settings_dict=settings_dict, input_table_aliases=["_ferc", "_eia"])

### Estimate Model Parameters
- We can estimate the m value with both labeled training data and with unsupervised EM method.

The u and m probabilities are parameters nested within each comparison level that can be viwed.

These m and u estimation functions update the values and return nothing.

The u parameters represent the proportion of record comparisons that fall into each comparison level amongst truly non-matching records.

In [ ]:
%%time
# run time goes down to 1 min with target_rows = 1e7
linker.estimate_u_using_random_sampling(target_rows=1e8)

Try with labeled data? Could try with the 2020 training data but there isn't very much of it so ideally this m value estimation is unsupervised.

In [ ]:
%%time
# feeding it a no impact blocking rule, or does this have any impact?
linker.estimate_parameters_using_expectation_maximisation("l.report_year == r.report_year")

The `u` values are the proportion of records falling into each ComparisonLevel amongst truly non-matching records.

The `m` values are the proportion of records falling into each ComparisonLevel amongst truly matching records.

TODO:
This isn't looking great: A much larger percentage of matches should have a match on `plant_name` or `utility_name`.
- Make the edit distance threshold larger for `plant_name` and `utility_name`. 
- Raise thresholds for `capacity_mw`?
- Do the levels need to be exclusive of each other? i.e. if `capacity_mw` diff < 10% it shouldn't fall into the <20% category as well?
- Add in columns / dictionaries to map to plant type.
- Try training with labels
- Mess with `term_frequency_adjustments`. Watch tutorial to see what they say about skew.

In [ ]:
linker.match_weights_chart()

In [ ]:
linker.m_u_parameters_chart()

In [ ]:
model_settings_name = "plant_gen_2020_0"

In [ ]:
settings = linker.save_settings_to_json(f"./splink_model_settings/{model_settings_name}.json", overwrite=False)

In [ ]:
# linker = DuckDBLinker([ferc_df, eia_df])
# linker.load_settings_from_json(f"./splink_model_settings/{model_settings_name}.json")

In [ ]:
%%time
df_preds = linker.predict()

Had trouble getting this as a pandas dataframe - df_preds.as_pandas_dataframe() without a limit set hung the kernel.

Look at source code to see what this function is actually doing.

In [ ]:
df_preds.as_pandas_dataframe(limit=5)

In [ ]:
# need to set retain_intermediate_calculations and retain_matching_columns to True in settings dictionary for prediction charts to work
records_to_view  = df_preds.as_record_dict(limit=5)
linker.waterfall_chart(records_to_view, filter_nulls=False)

In [ ]:
linker.comparison_viewer_dashboard(df_preds, "test.html", overwrite=True)

# You can view the scv.html file in your browser, or inline in a notbook as follows
from IPython.display import IFrame
IFrame(
    src="./test.html", width="100%", height=1200
)  
